### 线性规划与Word Mover's Distance

WMD在文本分析领域算作是一个比较经典的算法，它可以用来计算两个文本之间的相似度。 比如问答系统中，可以判断一个用户的query跟哪一个知识库里的问题最相近。而且，计算两个文本之间的相似度这个问题是NLP的核心，这也是为什么文本相似度计算这么重要的原因。 

背景： 在文本相似度匹配问题上如果使用tf-idf等模型，那这时候假如两个文本中没有出现共同的单词，则计算出来的相似度为0，但我们知道实际上很多时候单词可能不一样，但表示的内容确是类似的。 比如 ”People like this car“, "Those guys enjoy driving that", 虽然没有任何一样的单词，意思确是类似的。 这是WMD算法提出来的初衷。

WMD作为文本相似度计算的一种方法，最早由Matt J. Kusner, Yu Sun, Nicholas I. Kolkin, Kilian Q. Weinberger等人提出。但实际上它的想法极其简单，可以认为是Transportation Problem用在了词向量上， 其核心是线性规划。 对于Transportation问题在课上已经讲过，仍不清楚的朋友可以回顾一下课程的内容。 

在Section B里我们需要做两件事情： 1.  实现WMD算法来计算两个字符串之间的距离。  2. WMD的拓展方案

#### 1. WMD算法的实现
具体算法的实现是基于线性规划问题，细节请参考WMD的论文。 核心思想是把第一个句子转换成第二个句子过程中需要花费的最小cost。 

<img src="picture1.png" alt="drawing" width="600"/>


线性规划问题即可以写成如下形式：

<img src="picture2.png" alt="drawing" width="500"/>



这里的参数是$T_{ij}$， 需要通过LP Solver去解决。$c(i,j)$指的是两个单词之间的距离, $c_{i,j}=||x_i-x_j||_2$。 参考： $||x||_2=\sqrt{x_1^2+...+x_d^2}$

为了实现WMD算法，首先需要词向量。 在这里，我们就不自己去训练了，直接使用已经训练好的词向量。 
请下载训练好的Glove向量：https://nlp.stanford.edu/projects/glove/ 下载其中的 glove.6B.zip， 并使用d=100维的向量。 由于文件较大，需要一些时间来下载。 如果太慢的话，可以找找国内的镜像。

请注意：提交作业时不要上传此文件， 但文件路径请使用我们给定的路径，不要改变。 

In [8]:
import numpy
from numpy import *

In [9]:
import cvxopt
from cvxopt import matrix
# cvxopt.solvers.lp()

In [10]:
# 读取Glove文件。 注意： 不要试图修改文件以及路径
import numpy as np
glovefile = open("glove.6B/glove.6B.100d.txt","r",encoding="utf-8") 
dict={}
def get_coefs(word, *arr):#获取glove数据集中的数据，每一行的形式为word,embedding1...embedding300
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    for line in path.readlines(): 
        a=line.strip().split(' ')
        value=np.asarray(a[1:], dtype='float32')
        dict[a[0]]=value

#     with open(path,encoding='utf8') as f:
#         return dict(get_coefs(*line.strip().split(' ')) for line in f)

load_embeddings(glovefile)

In [11]:
for key in dict.keys():
    print(key)
    print(dict[key])
    break

the
[-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.270

In [12]:
numpy.sqrt(numpy.sum(numpy.square([3,2])))

3.605551275463989

In [60]:
# 读取Glove文件。 注意： 不要试图修改文件以及路径
# glovefile = open("glove.6B.100d.txt","r",encoding="utf-8")  

# TODO: 编写WMD函数来计算两个句子之间的相似度

def WMD (sent1, sent2):
    """
    这是主要的函数模块。参数sent1是第一个句子， 参数sent2是第二个句子，可以认为没有经过分词。
    在英文里，用空格作为分词符号。
    
    在实现WMD算法的时候，需要用到LP Solver用来解决Transportation proboem. 请使用http://cvxopt.org/examples/tutorial/lp.html
    也可以参考blog： https://scaron.info/blog/linear-programming-in-python-with-cvxopt.html
    
    需要做的事情为：
    
    1. 对句子做分词： 调用 .split() 函数即可
    2. 获取每个单词的词向量。这需要读取文件之后构建embedding matrix. 
    3. 构建lp问题，并用solver解决
    
    可以自行定义其他的函数，但务必不要改写WMD函数名。测试时保证WMD函数能够正确运行。
    """
    sent0_list=sent1.split();
    sent1_list=sent2.split();
    d0_word=[]
    d1_word=[]
    d0_num={}
    d1_num={}
    count=0
    d0=[]
    d1=[]
    c=[]
    d=[]
    for word in sent0_list:
        if word not in d0_word:
            d0_word.append(word)
            d0_num[word]=1
        else:
            d0_num[word]=d0_num[word]+1
            
    for word in d0_word:    
        d0.append(d0_num[word]/len(d0_word))
        
    for word in sent1_list:
        if word not in d1_word:
            d1_word.append(word)
            d1_num[word]=1
        else:
            d1_num[word]=d1_num[word]+1
            
    for word in d1_word:    
        d1.append(d1_num[word]/len(d1_word))      
        
    for i in range(len(d0_word)):
        for j in range(len(d1_word)):
            a0=dict[d0_word[i]]
            a1=dict[d1_word[j]]
            c.append(numpy.sqrt(numpy.sum(numpy.square(a0-a1))))
    for i in range(len(c)):
        c[i]=float(c[i])
#         生成d0 d1矩阵 并将两个矩阵合并
    d0_m=numpy.zeros((len(d0_word),len(d0_word)*len(d1_word)),dtype=float)
    for i in range(len(d0_word)):
        for j in range(len(d1_word)):
            d0_m[i][i*len(d1_word)+j]=1
#     print(d0_m)

    d1_m=numpy.zeros((len(d1_word),len(d0_word)*len(d1_word)),dtype=float)
    for i in range(len(d1_word)):
        for j in range(len(d0_word)):
            d1_m[i][j*len(d1_word)+i]=1
#     print(d1_m)
    d01_merge=np.vstack((d0_m,d1_m))
#     print(len(d01_merge))
#     print(d01_merge)
#  d
    d01_merge=delete(d01_merge,len(d01_merge)-1,axis=0)
#     print(len(d01_merge))
    d=d0+d1
    del(d[-1])
    
    print(type(d))
    d_m=numpy.zeros((len(d0_word)*len(d1_word),len(d0_word)*len(d1_word)),dtype=float)    
    for i in range(len(d0_word)*len(d1_word)):
        for j in range(len(d0_word)*len(d1_word)):
            if(i==j):
                d_m[i][j]=-1
    
    h_m=[float64(0)]*(len(d0_word)*len(d1_word))
#     print(h_m)
#     print(d_m)
#     c = matrix([1.0,2.0])
    c=matrix(c)
#     print(c)
    G=matrix(d_m)
#     print(G)
#     print(G)
    h=matrix(h_m)
#     print(h)
#     print(type(d_m))

    A=matrix(d01_merge)
#     print(A)
    b=matrix(d)
#     print(b)
#     c.astype(float)
#     print(type(c[0]))
#     print(c)
#     return wmd_dist
    sv=cvxopt.solvers.lp(c,G,h,A,b)    
    print(sv['x'])
    print(np.dot(sv['x'].T,c))

In [70]:
## TODO: 自己写至少4个Test cases来测试一下。 比如 print (WMD("people like this car", "those guys enjoy driving that"))
##       
# WMD('i like sports','i want to play basketball')
# WMD('this book is very good','i want to play basketball')
# WMD('this book is very good','this book is very good')
# WMD('i want  basketball','i like sports')
# WMD('i want  basketball','are you kidding me')
# WMD('this is so fast','this is so fast')
# print(sv['x'])

<class 'list'>
     pcost       dcost       gap    pres   dres   k/t
 0:  5.4591e+00  5.4591e+00  2e+00  2e-16  3e-01  1e+00
 1:  5.3665e+00  5.3346e+00  4e-01  8e-17  7e-02  2e-01
 2:  5.1799e+00  5.1495e+00  1e-01  3e-16  2e-02  4e-02
 3:  5.1148e+00  5.1059e+00  3e-02  8e-16  3e-03  1e-03
 4:  5.0980e+00  5.0977e+00  9e-04  1e-16  1e-04  4e-05
 5:  5.0974e+00  5.0974e+00  9e-06  2e-16  1e-06  4e-07
 6:  5.0974e+00  5.0974e+00  9e-08  1e-16  1e-08  4e-09
Optimal solution found.
[ 4.30e-09]
[ 5.53e-09]
[ 2.00e-01]
[ 8.25e-09]
[ 3.47e-09]
[ 5.59e-09]
[ 5.46e-09]
[ 5.02e-09]
[ 2.54e-08]
[ 2.00e-01]
[ 6.43e-09]
[ 2.25e-08]
[ 1.07e-08]
[ 2.00e-01]
[ 2.93e-08]
[ 2.00e-01]
[ 3.93e-08]
[ 3.51e-09]
[ 2.66e-08]
[ 4.39e-09]
[ 5.75e-08]
[ 2.00e-01]
[ 2.33e-09]
[ 8.66e-09]
[ 4.32e-09]

[[5.09742556]]


In [43]:
print

<function print>

In [130]:
print(matrix([1,2],(1,2)))
print(matrix([1,2]))

[ 1  2]

[ 1]
[ 2]



In [136]:
print(matrix([[1,2],[1,2]],(2,2)))

TypeError: wrong matrix dimensions

In [ ]:
import pulp
import numpy as np
from pprint import pprint

def transportation_problem(costs, x_max, y_max):

    row = len(costs)
    col = len(costs[0])

    prob = pulp.LpProblem('Transportation Problem', sense=pulp.LpMaximize)

    var = [[pulp.LpVariable(f'x{i}{j}', lowBound=0, cat=pulp.LpInteger) for j in range(col)] for i in range(row)]

    flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]

    prob += pulp.lpDot(flatten(var), costs.flatten())

    for i in range(row):
        prob += (pulp.lpSum(var[i]) <= x_max[i])

    for j in range(col):
        prob += (pulp.lpSum([var[i][j] for i in range(row)]) <= y_max[j])

    prob.solve()

    return {'objective':pulp.value(prob.objective), 'var': [[pulp.value(var[i][j]) for j in range(col)] for i in range(row)]}

#### 2. WMD算法的拓展


##### 2.1 从欧式距离到Mahalanobis距离
WMD算法本身不需要任何的标注好的数据，所以它属于无监督学习。 而且在上述的WMD算法里使用的是欧式距离，$c(i,j)=||x_i-x_j||_2$， 那这种距离有什么缺点呢？ 其中一个缺点是欧式距离的计算把一个空间里的每一个维度都看成了同样的权重，也就是每一个维度的重要性都是一致的，而且不同维度之间的相关性也没有考虑进来。如果想把这些信息考虑进来，我们则可以使用一个改进版的距离计算叫做Mahalanobis Distance， 距离计算变成 $c(i,j)=(x_i-x_j)^{\top}M(x_i-x_j)$。

这如何去理解呢？ Mahalanobis distance可以理解成： 首先我们对原始空间里的样本做了一层线性的转换， 然后在转换后的空间里计算欧式距离。 我们把这个过程写一下： 原始空间里的点为 $x_i$, 然后我们定义一个转换矩阵 $L$, 这时候就可以得到 $||Lx_i - Lx_j||_2^2=||L(x_i-x_j)||_2^2=(L(x_i-x_j))^{\top}L(x_i-x_j)=(x_i-x_j)^{\top}L^{\top}L(x_i-x_j)=(x_i-x_j)^{\top}M(x_i-x_j)$, 相当于把$L^{\top}L$看做是矩阵$M$。这时候很容易看出来矩阵$M$是PSD(positive semidefinite). 

假设我们定义了这种距离，这里的M如何选择呢？ 当然，这是需要学出来的！ 那为了学出M， 必须要有标注好的训练数据，也就需要监督学习场景！  

##### 2.2 从无监督学习到监督学习

假如拥有数据集$D={(s_1, y_1),...,(s_n, y_n)}$， 这里每一个$s_i$代表的是一个句子， $y_i$代表的是对应每一个句子的标签（label)。 我们希望使用这个数据来学出M的值。那如何学习呢？ 在这个问题上能使用的方法其实比较多，但在这里， 我们采用一个margin-based方法，这一点在SVM里面其实接触过。

具体一点，假如我们手里有三个句子，$s_u, s_v, s_w$, 其中$s_u$和$s_v$是属于同一个类别，$s_w$是属于另一个类别，那这时候从KNN的角度来讲，我们希望$s_u, s_v$的距离要小于 $s_u, s_w$之间的距离。 用数学来表示： $d(s_u, s_v) < d(s_u, s_w)$, $~~d(.,.)$表示两个文本之间的距离。 其实我们希望它们之间的距离越大越好，也就是所谓的完全区间越宽越好。 但实际上，这个距离太大也没有什么意义，所以我们就干脆指定一个参数 $\eta$来表示margin, 也就是只要它俩之间的距离大于这个margin就可以。如果小于margin就给他们一些惩罚（penalty)，这一点跟SVM极其相似（slack variable）。所以从这个角度SVM也叫做margin-based classifier. 

把上述的表示成数学的话： $d(s_u, s_v) + \eta < d(s_u, s_k)$, 但如果这个式子不成立的话就可以认为产生了penalty。 所以这部分就可以表示成大家熟悉的hinge loss:    $max (0,  d(s_u, s_v) + \eta - d(s_u, s_k))$。 另外，我们同时也希望如果两个样本属于同一个类别， 那它俩的距离也比较相近。所以目标函数可以分为两个部分： 1. 同类型的样本距离尽量要近   2. 不同类型的样本距离尽量远一些。 

当我们把所有的样本以及他们之间的大小关系考虑进来之后就可以得到最终的目标函数。 

\begin{equation}
L = \lambda \sum_{u=1}^{n}\sum_{v\in pos(u)}d(s_u, s_v) + (1-\lambda)\sum_{u=1}^{n}\sum_{v\in pos(u)}^{}\sum_{w\in neg(u)}^{} max (0,  d(s_u, s_v) + \eta - d(s_u, s_w))
\end{equation}

这里几个notation:  pos(u)代表的是跟样本u属于同一个类别的样本， neg(u)指的是跟样本u属于不同类别的样本。 注意：类别的个数可以大于2, 就是多分类问题。 你也可以参考： http://jmlr.org/papers/volume10/weinberger09a/weinberger09a.pdf

在这个式子里，第一部分代表的是让同一类型的样本的距离变小， 第二部分代表的是不同类型的样本之间要扩大距离。 

- #### Q1: 这里$\lambda$起到什么作用？

// TODO:  你的答案....




- #### Q2: 在目标函数里有$\eta$值，这个值怎么理解？ 如果去设定这个值呢？


// TODO: 你的答案 ....
    
    
    
    
    

这里的$d_{u,v}$指的是$s_u$和$s_v$之间的距离， 而且这个距离被定义为：

\begin{equation} d_{u, v}=min_{T\geq 0}\sum_{i,j}^{}T_{ij}c(i,j)^u~~~~  s.t. \sum_{j=1}^{}T_{ij}=d_i^u, ~~\sum_{i=1}^{}T_{ij}=d_j'^v\end{equation}

这里  $c(i,j)=(x_i-x_j)^{\top}M(x_i-x_j)$。 所以是不是可以察觉到这个问题目标函数里既包含了参数$M$也包含了线性规划问题。

- #### Q3: 请试着去理解上述所有的过程，并回答： 优化问题如何解决呢？ 请给出解题的思路 （文字适当配合推导过程）。 

// TODO 你的答案.... 

对于上述问题，其实我们也可以采用不一样的损失函数来求解M。 一个常用的损失函数叫作 “kNN-LOO error”， 相当于把KNN的准确率转换成了smooth differential loss function. 感兴趣的朋友可以参考： https://papers.nips.cc/paper/6139-supervised-word-movers-distance.pdf



以上是优化部分的一个简短的作业，通过这些练习会对优化理论有更清晰的认知。  Good luck for everyone！ 